# 데이터 전처리 - 문서 검색 시스템

# pdf

In [1]:
import os
import pdfplumber
import re
import tabula
import pandas as pd
import nltk
import openai
from dotenv import load_dotenv
import requests
import json
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [12]:
def extract_text_and_tables_to_dataframe(pdf_path):
    page_contents = []
    text1 = ""  # text1 변수 초기화

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            # 테이블 추출
            tables = page.extract_tables()

            # 테이블 데이터프레임으로 변환
            table_dfs = []
            if tables:
                for table_num, table in enumerate(tables):
                    df = pd.DataFrame(table[1:], columns=table[0])  # 첫 번째 행을 헤더로 사용
                    table_dfs.append(df)

            # 텍스트 추출
            text_content = page.extract_text()

            # 텍스트에서 테이블의 내용 제거
            if tables and text_content:
                for df in table_dfs:
                    table_text = dataframe_to_markdown(df)
                    text_content = text_content.replace(table_text, '')

            # 페이지 내용 추가
            page_contents.append({'Page': i + 1, 'Text': text_content.strip() if text_content else None, 'Tables': table_dfs})

            # text1에 텍스트 및 테이블 데이터 추가
            if text_content:
                text1 += text_content + "\n\n"  # 각 페이지 사이에 개행 추가

                # 테이블 데이터도 추가
                for df in table_dfs:
                    text1 += "```\n"  # 테이블 시작
                    text1 += dataframe_to_markdown(df) + "\n"
                    text1 += "```\n\n"  # 테이블 끝

    return page_contents, text1

def dataframe_to_markdown(df):
    # 데이터프레임을 Markdown 표 형식으로 변환
    return df.to_markdown(index=False)

# PDF 파일에서 텍스트와 테이블 추출하여 데이터프레임으로 변환
path = "./manual/OT_훈련생 안내사항_최종본 2024.06.24.pdf"
page_contents, text1 = extract_text_and_tables_to_dataframe(path)

# 추출된 페이지 내용 출력
for content in page_contents:
    page_num = content['Page']
    text_content = content['Text']
    tables = content['Tables']
    
    if text_content:
        print(f"Page {page_num} Text:")
        print(text_content)
    
    if tables:
        for table_num, df in enumerate(tables):
            print(f"Page {page_num} Table {table_num + 1} (Markdown format):")
            markdown_table = dataframe_to_markdown(df)
            print("```\n" + markdown_table + "\n```")
    
    print()

# "■|□" 패턴을 기준으로 텍스트를 나누기
pattern = r"(■|□)\s*(.*)"
matches = re.findall(pattern, text1)

# 각 조항을 제목과 내용으로 분리하여 리스트에 추가
split_text1 = []

for i in range(len(matches)):
    section_title = matches[i][0] + " " + matches[i][1].strip()  # 섹션 제목 (matches[i]의 첫 번째 요소는 "■" 또는 "□")
    if i < len(matches) - 1:
        start_index = text1.find(matches[i][0] + " " + matches[i][1])
        end_index = text1.find(matches[i + 1][0] + " " + matches[i + 1][1])
        section_content = text1[start_index:end_index].strip()
    else:
        start_index = text1.find(matches[i][0] + " " + matches[i][1])
        section_content = text1[start_index:].strip()  # 마지막 섹션의 경우 끝까지 추출

    split_text1.append((section_title, section_content))

# 각 섹션의 내용에서 일반 텍스트의 개행문자 제거 및 (p.n참조) 패턴 제거
documents = []

for section_title, section_content in split_text1:
    # 섹션 제목
    section_title_clean = re.sub(r'[■□➢*]', '', section_title).strip()  # 특수기호 제거

    # 섹션 내용 (테이블 부분의 개행 문자는 유지하고, 일반 텍스트 부분의 개행 문자는 제거)
    section_content_parts = section_content.split('```')
    cleaned_parts = []
    for i, part in enumerate(section_content_parts):
        if i % 2 == 0:  # 일반 텍스트 부분
            cleaned_parts.append(re.sub(r'[■□➢*]', '', part).replace('\n', '').strip())  # 특수기호 제거
        else:  # 테이블 부분
            cleaned_parts.append(part.strip())

    section_content_clean = '```'.join(cleaned_parts)

    # 섹션 제목과 내용이 동일하고 "(p.n참조)"를 포함하면 삭제
    if section_title_clean == section_content_clean and re.search(r'\(p\.\d+참조\)', section_content_clean):
        continue

    # JSON 형식으로 변환하여 저장
    document = {"section": section_title_clean, "content": section_content_clean}
    documents.append(document)

# JSON 파일로 저장
with open('OT_훈련생 안내사항_최종본 2024.06.24.json', 'w', encoding='utf-8') as f:
    json.dump(documents, f, ensure_ascii=False, indent=4)

print("JSON 파일로 저장되었습니다.")


Page 1 Text:
Orientation 자료
■ 출결관리
: 입퇴실시, 입실-퇴실 시간에 맞춰 hrd-net 비콘을 통해 출결체크를 해야 합니다.
➢ 출결 방법
1) hrd-net 앱 다운로드 및 로그인
2) hrd-net 앱 출결 체크 하단 비콘 클릭 (입실 시간 체크 확인 필수)
** 훈련생의 부주의로 입실 또는 퇴실 체크를 하지 않으면 당일 결석으로 처리되며, 해당 사항은 본인의
책임입니다.
■ 내일배움카드 기준 수료 기준 및 출결관리 (p.3참조)
■ 훈련수당 (p.5참조)
■ 점심 식사
1) 각 기수 학생 대표가 결석자, 휴가, 기타 등 제외한 인원 파악
2) 오전 11시까지 사무실 방문 후 식권대장에 체크
3) 식권 일괄 수령 후 구내식당에 식권 제출 후 맛있는 점심 식사하기(갑을그레이트밸리 11:50 ~
12:50)
※ 구내식당 장소: 갑을그레이트밸리 지하 1층.
■ 강의실 내 음식물 섭취 시
-강의장 내 음료 제외한 음식물 섭취는 자제해주시고, 쓰레기는 복도, 라운지, 로비 등에 비치된 분리수거
함에 버려 주시기 바랍니다. 또한, 라운지에서 라면, 도시락과 같은 음식 섭취는 불가합니다.
■ 건물 내 흡연 절대 금지 (과태료), 화물용 엘리베이터 탑승 금지
■ 개인 사물함
-사용기한: 교육시작일 ~ 교육수료일 *교육수료이후 사물함 물건은 폐기처리 예정입니다.
-사물함 열쇠 소지자는 교육 종료 시 반납하고, 열쇠 분실 책임은 본인에게 있음.
■ 방과후 스터디
1) 개방 시간: 수업 종료 ~ 21:00 (주말 및 공휴일 불가)
1

Page 2 Text:
* 21시부터는 마리오 아울렛 보안관제센터에서 전체 소등 예정
* 출입문에 대한 관리가 제대로 되지 않을 경우, 6시 30분 이후로는 센터를 개방하지 않을 예정이오니
교육생분들은 반드시 해당 내용을 준수해주시기 바랍니다.
2) 석식제공: 오후 5시 50분~6시 20분, 사무실 방문하여 훈련생 식대 장부 작성 후 수령
* 스터디 진행시 자리 정리, 정숙 등 타 강의장에 방해되지 않도록 주의 바랍니

In [34]:
import pdfplumber
import pandas as pd
import re
import json

def extract_text_and_tables_to_dataframe(pdf_path):
    page_contents = []
    text1 = ""  # text1 변수 초기화

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            # 테이블 추출
            tables = page.extract_tables()

            # 테이블 데이터프레임으로 변환
            table_dfs = []
            if tables:
                for table_num, table in enumerate(tables):
                    df = pd.DataFrame(table[1:], columns=table[0])  # 첫 번째 행을 헤더로 사용
                    table_dfs.append(df)

            # 텍스트 추출
            text_content = page.extract_text()

            # 텍스트에서 테이블의 내용 제거
            if tables and text_content:
                for df in table_dfs:
                    table_text = dataframe_to_markdown(df)
                    text_content = text_content.replace(table_text, '')

            # 페이지 내용 추가
            page_contents.append({'Page': i + 1, 'Text': text_content.strip() if text_content else None, 'Tables': table_dfs})

            # text1에 텍스트 및 테이블 데이터 추가
            if text_content:
                text1 += text_content + "\n\n"  # 각 페이지 사이에 개행 추가

                # 테이블 데이터도 추가
                for df in table_dfs:
                    text1 += "```\n"  # 테이블 시작
                    text1 += dataframe_to_markdown(df) + "\n"
                    text1 += "```\n\n"  # 테이블 끝

    return page_contents, text1

def dataframe_to_markdown(df):
    # 데이터프레임을 Markdown 표 형식으로 변환
    return df.to_markdown(index=False)

def clean_text(text):
    # 특수문자 제거
    return re.sub(r'[■□ㅇ▶※]', '', text).replace('\n', '').strip()  # 특수기호 제거

def split_text_by_pattern(text, pattern):
    matches = list(re.finditer(pattern, text))
    split_text = []

    if matches:
        for i in range(len(matches)):
            section_title = clean_text(matches[i].group().strip())
            if i < len(matches) - 1:
                section_content = clean_text(text[matches[i].end():matches[i + 1].start()].strip())
            else:
                section_content = clean_text(text[matches[i].end():].strip())
            
            split_text.append((section_title, section_content))
    else:
        split_text.append(("전체 텍스트", clean_text(text.strip())))
    
    return split_text

def save_to_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# PDF 파일에서 텍스트와 테이블 추출하여 데이터프레임으로 변환
path = "./manual/[별지 2] 개인정보의 수집&이용 및 제공에 관한 동의서_20231228수정.pdf"
page_contents, text1 = extract_text_and_tables_to_dataframe(path)

# 추출된 페이지 내용 출력
for content in page_contents:
    page_num = content['Page']
    text_content = content['Text']
    tables = content['Tables']
    
    if text_content:
        print(f"Page {page_num} Text:")
        print(text_content)
    
    if tables:
        for table_num, df in enumerate(tables):
            print(f"Page {page_num} Table {table_num + 1} (Markdown format):")
            markdown_table = dataframe_to_markdown(df)
            print("```\n" + markdown_table + "\n```")
    
    print()

# "■|□|▶|※" 패턴을 기준으로 텍스트를 나누기
pattern = r"(▶|※)\s*(.*)"
split_text = split_text_by_pattern(text1, pattern)

# 각 섹션을 JSON 형식으로 변환하여 저장
documents = [{"section": title.strip(), "content": f"{title.strip()} {content.strip()}"} for title, content in split_text]

# JSON 파일로 저장
save_to_json(documents, '[별지 2] 개인정보의 수집&이용 및 제공에 관한 동의서_20231228수정.json')

# 결과 출력 (선택 사항)
for doc in documents:
    print(f"제목: {doc['section']}")
    print(f"내용: {doc['content']}")
    print("===================================")

print("JSON 파일로 저장되었습니다.")


Page 1 Text:
■ 국민내일배움카드 운영규정 [별지 제2호서식]
개인정보의 수집ㆍ이용 및 제공에 관한 동의서
▶ 개인정보 수집ㆍ이용에 관한 안내
ㅇ 고용노동부는 직업능력개발계좌 발급 및 지원 등에 관한 사무를 수행
하기 위하여 「국민 평생 직업능력 개발법 시행령」 제52조의2제1항에
따라 주민등록번호 또는 외국인등록번호가 포함된 자료 및 기타 민감정
보를 다음과 같이 직업능력개발정보망(HRD-Net)에 수집ㆍ관리하고 있으며,
「개인정보 보호법」에서 규정하고 있는 책임과 의무를 준수하고 실천합니
다.
수집목적 수집항목 보유기간
1) 직업능력개발계좌 발급 및 훈련 진단‧상
담
2) 비용지원 및 개인별 훈련이력 관리 성명, 주민등록번호, 주소, 이메일주소
3) 일자리정보, 취업지원사업 등 관련정보 (선택), 전화번호 또는 휴대전화번호, 수집일부터
및 서비스의 제공 그 밖에 학력･자격･경력 등 상담 및 계속
4) 정부의 직업능력개발훈련 성과관리 계좌발급․지원에 필요한 민감정보
5) 훈련 만족도 조사 및 제도개선에 필요한
의견조사 실시 등
ㅇ 고용노동부는 개인정보를 수집목적에 필요한 범위에서 적합하게 처리
하며, 정보주체의 사전 동의 없이 개인정보를 제3자에게 제공하지 않
습니다.
ㅇ 직업능력개발계좌를 발급받은 사람(정보주체)은 자신에 관한 개인정보
의 열람 ․ 수정을 요청할 수 있습니다. 다만, 훈련실시 및 비용지원 등
에 관한 정보는 오류가 있는 경우를 제외하고는 수정․삭제를 요청할 수
없습니다.
▶ 개인정보의 제공에 관한 동의
ㅇ 고용노동부는 취업지원 관련 정보 및 서비스 제공 등을 위하여 직업능력개발계
좌를 발급받은 사람(정보주체)으로부터 제공받은 개인정보를 다음과 같이
제공(보유기관으로부터 수집ㆍ제공 포함)합니다.
Page 1 Table 1 (Markdown format):
```
| 수집목적                                 | 수집항목                             | 보유기간   |
|:----------

# pdf 후처리

In [18]:
def extract_text_clean(pdf_path):
    full_text = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            width, height = page.width, page.height
            
            left_bbox = (0, 0, width * 0.5, height)
            right_bbox = (width * 0.5, 0, width, height)
            
            left_text = page.within_bbox(left_bbox).extract_text()
            right_text = page.within_bbox(right_bbox).extract_text()
            
            if left_text:
                full_text.append(left_text)
            if right_text:
                full_text.append(right_text)
    
    return "\n".join(full_text)

def split_text_by_pattern(text, pattern):
    matches = re.findall(pattern, text)
    split_text = []
    
    # 처음 나오는 문자열을 별도로 처리
    if matches:
        first_match_pos = text.find(matches[0])
        intro_text = text[:first_match_pos].strip()
        
        if intro_text:
            # 제목에서 첫 번째 줄만 추출
            intro_title = intro_text.split('\n')[0]
            split_text.append((intro_title, intro_text))
        
        for i in range(len(matches)):
            section_title = matches[i].strip()
            if i < len(matches) - 1:
                section_content = text.split(section_title, 1)[-1].split(matches[i + 1], 1)[0].strip()
            else:
                section_content = text.split(section_title, 1)[-1].strip()
            
            # 제목에서 괄호 안의 문자열만 추출
            title_inside_parentheses = re.search(r"\((.*?)\)", section_title).group(1)
            
            # section 제목을 content의 앞에 추가
            combined_content = f"{section_title} {section_content}"
            split_text.append((title_inside_parentheses, combined_content))
    else:
        split_text.append(("전체 텍스트", text.strip()))
    
    return split_text

def save_to_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def remove_last_n_characters(text, n):
    """텍스트의 마지막 n 문자를 제거"""
    return text[:-n] if n < len(text) else ""

if __name__ == "__main__":

    # 텍스트 추출
    path = "./manual/훈련생준수사항_240624.pdf"
    text = extract_text_clean(path)
    
    # 텍스트의 마지막 n 문자 제거 (예: 마지막 1000 문자 제거)
    n = 85
    text = remove_last_n_characters(text, n)
    
    # "제 n조(문자열)" 패턴을 기준으로 텍스트를 나누기
    pattern = r"(제\s?\d+\s*조\s*\([^\)]+\))"
    split_text = split_text_by_pattern(text, pattern)
    
    # 각 섹션을 JSON 형식으로 변환
    documents = [{"section": title.strip(), "content": content.replace("\n", "").strip()} for title, content in split_text]
    
    # JSON 파일로 저장
    save_to_json(documents, '훈련생준수사항_240624.json')
    
    # 결과 출력 (선택 사항)
    for doc in documents:
        print(f"제목: {doc['section']}")
        print(f"내용: {doc['content']}")
        print("===================================")

print("JSON 파일로 저장되었습니다.")


제목: 천재 IT교육센터 생활 규정
내용: 천재 IT교육센터 생활 규정시행 2023.06.29. 일부개정 2023.12.08.2차 개정 2023.12.19.3차 개정 2024.04.19.4차 개정 2024.06.24.
제목: 목적
내용: 제1조(목적) 이 규정은 천재 IT교육센터 훈련생의 훈련생활과 관련된 준수사항을 규정함으로써 공정하고 일관된 기준에 따라 훈련생의 효과적인 학습 및 협업을 실현하며, 교육 효율성과 질서를 유지하기위한 것을 목적으로 합니다.
제목: 수강태도
내용: 제 2조(수강태도) ① 수업 시작 시간인 9시 전에 착석해주시기 바랍니다. 9시 전 입실 및 수업에 대한 준비는 강사와의상호예의이자 가장 기본적인 룰입니다. 9시 전 입실하여 수업 준비를 완료하셔서 다른 교육생들에 주는 피해를 삼가주시기 바랍니다.➁ 수업시간에 성실한 태도를 유지합니다. 휴가, 결석, 지각, 조퇴, 외출 등은 사전에 허가를 받고 실시합니다. 쉬는 시간 이외의 수업 도중 또는 프로젝트 기간 중에는 무단 이탈을 금지합니다.➂ 다른 훈련생들의 학습 환경을 존중하기 위해 소음을 최소화합니다. 강의실 내에서는 조용한 분위기를 유지해야 합니다.④ 훈련교사의 수업방식을 존중하고, 훈련교사의 지도에 따라 훈련에 적극적으로 참여해주시기 바랍니다.⑤ 수업 시간은 정각 ~ 50분 진행 후 10분간 쉬는 시간입니다. 쉬는 시간 외에 강의실 이탈 시 경고 1회가 주어지며, 3회 이후 제적 대상임을 알려드립니다.
제목: 타인 존중
내용: 제 3조(타인 존중) ① 타인을 존중하고 예의를 갖추며, 언어와 행동에 신중함을 기합니다. 다른 훈련생을 비방하거나 인신공격 등의 행위는 감점 요인에 해당될 수 있습니다.➁ 교육기간 동안에 기본 매너를 지키지 않아 훈련생 간에 갈등이 발생시 학습 분위기 저해로 인해 타훈련생에게 피해가 돌아갈 수 있으니 유의해주시기 바랍니다.
제목: 협력과 커뮤니케이션
내용: 제 4조(협력과 커뮤니케이션) 훈련생들 간의 협력과 원활한 커뮤니케이션을 지향합니다. 서로를 도와주고 

In [ ]:
import pdfplumber
import pandas as pd
import re
import json

def extract_text_and_tables_to_dataframe(pdf_path):
    page_contents = []
    text1 = ""  # text1 변수 초기화

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            # 테이블 추출
            tables = page.extract_tables()

            # 테이블 데이터프레임으로 변환
            table_dfs = []
            if tables:
                for table_num, table in enumerate(tables):
                    df = pd.DataFrame(table[1:], columns=table[0])  # 첫 번째 행을 헤더로 사용
                    table_dfs.append(df)

            # 텍스트 추출
            text_content = page.extract_text()

            # 텍스트에서 테이블의 내용 제거
            if tables and text_content:
                for df in table_dfs:
                    table_text = dataframe_to_markdown(df)
                    text_content = text_content.replace(table_text, '')

            # 페이지 내용 추가
            page_contents.append({'Page': i + 1, 'Text': text_content.strip() if text_content else None, 'Tables': table_dfs})

            # text1에 텍스트 및 테이블 데이터 추가
            if text_content:
                text1 += text_content + "\n\n"  # 각 페이지 사이에 개행 추가

                # 테이블 데이터도 추가
                for df in table_dfs:
                    text1 += "```\n"  # 테이블 시작
                    text1 += dataframe_to_markdown(df) + "\n"
                    text1 += "```\n\n"  # 테이블 끝

    return page_contents, text1

def dataframe_to_markdown(df):
    # 데이터프레임을 Markdown 표 형식으로 변환
    return df.to_markdown(index=False)

def clean_text(text):
    # 특수문자 제거
    return re.sub(r'[■□ㅇ▶※]', '', text).replace('\n', '').strip()  # 특수기호 제거

def split_text_by_pattern(text, pattern):
    matches = list(re.finditer(pattern, text))
    split_text = []

    if matches:
        for i in range(len(matches)):
            section_title = clean_text(matches[i].group().strip())
            if i < len(matches) - 1:
                section_content = clean_text(text[matches[i].end():matches[i + 1].start()].strip())
            else:
                section_content = clean_text(text[matches[i].end():].strip())
            
            split_text.append((section_title, section_content))
    else:
        split_text.append(("전체 텍스트", clean_text(text.strip())))
    
    return split_text

def save_to_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# PDF 파일에서 텍스트와 테이블 추출하여 데이터프레임으로 변환
path = "./manual/[별지 2] 개인정보의 수집&이용 및 제공에 관한 동의서_20231228수정.pdf"
page_contents, text1 = extract_text_and_tables_to_dataframe(path)

# 추출된 페이지 내용 출력
for content in page_contents:
    page_num = content['Page']
    text_content = content['Text']
    tables = content['Tables']
    
    if text_content:
        print(f"Page {page_num} Text:")
        print(text_content)
    
    if tables:
        for table_num, df in enumerate(tables):
            print(f"Page {page_num} Table {table_num + 1} (Markdown format):")
            markdown_table = dataframe_to_markdown(df)
            print("```\n" + markdown_table + "\n```")
    
    print()

# "■|□|▶|※" 패턴을 기준으로 텍스트를 나누기
pattern = r"(▶|※)\s*(.*)"
split_text = split_text_by_pattern(text1, pattern)

# 각 섹션을 JSON 형식으로 변환하여 저장
documents = [{"section": title.strip(), "content": f"{title.strip()} {content.strip()}"} for title, content in split_text]

# JSON 파일로 저장
save_to_json(documents, '[별지 2] 개인정보의 수집&이용 및 제공에 관한 동의서_20231228수정.json')

# 결과 출력 (선택 사항)
for doc in documents:
    print(f"제목: {doc['section']}")
    print(f"내용: {doc['content']}")
    print("===================================")

print("JSON 파일로 저장되었습니다.")


### 테이블 토큰수, 마크다운으로 통으로 가져오지 않고 테이블에서 원하는 내용만 추출할 수 있게. output으로 표 자체를 통으로 안내니까 상관없나? 확인해보기

In [27]:
import pdfplumber
import pandas as pd
import re
import json

def extract_text_and_tables_to_dataframe(pdf_path):
    page_contents = []
    text1 = ""  # text1 변수 초기화

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            # 테이블 추출
            tables = page.extract_tables()

            # 테이블 데이터프레임으로 변환
            table_dfs = []
            if tables:
                for table_num, table in enumerate(tables):
                    df = pd.DataFrame(table[1:], columns=table[0])  # 첫 번째 행을 헤더로 사용
                    table_dfs.append(df)

            # 텍스트 추출
            text_content = page.extract_text()

            # 특정 텍스트 패턴 제거
            if text_content:
                text_content = remove_specific_text(text_content)

            # 텍스트에서 테이블의 내용 제거
            if tables and text_content:
                for df in table_dfs:
                    table_text = dataframe_to_markdown(df)
                    text_content = text_content.replace(table_text, '')

            # 페이지 내용 추가
            page_contents.append({'Page': i + 1, 'Text': text_content.strip() if text_content else None, 'Tables': table_dfs})

            # text1에 텍스트 및 테이블 데이터 추가
            if text_content:
                text1 += text_content + "\n\n"  # 각 페이지 사이에 개행 추가

                # 테이블 데이터도 추가
                for df in table_dfs:
                    text1 += "```\n"  # 테이블 시작
                    text1 += dataframe_to_markdown(df) + "\n"
                    text1 += "```\n\n"  # 테이블 끝

    return page_contents, text1

def remove_specific_text(text):
    # 특정 텍스트 정규식으로 제거
    specific_text = "2024년 월 일위 개인정보의 수집 이용 안내 및 제공에 동의합니다\. 성명: \(서명\)"
    return re.sub(specific_text, '', text)

def dataframe_to_markdown(df):
    # 데이터프레임을 Markdown 표 형식으로 변환
    return df.to_markdown(index=False)

def clean_text(text):
    # 특수문자 제거
    return re.sub(r'[■□ㅇ▶※]', '', text).replace('\n', '').strip()  # 특수기호 제거

def split_text_by_pattern(text, pattern):
    matches = list(re.finditer(pattern, text))
    split_text = []

    if matches:
        for i in range(len(matches)):
            section_title = clean_text(matches[i].group().strip())
            if i < len(matches) - 1:
                section_content = clean_text(text[matches[i].end():matches[i + 1].start()].strip())
            else:
                section_content = clean_text(text[matches[i].end():].strip())
            
            split_text.append((section_title, section_content))
    else:
        split_text.append(("전체 텍스트", clean_text(text.strip())))
    
    return split_text

def save_to_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# PDF 파일에서 텍스트와 테이블 추출하여 데이터프레임으로 변환
path = "./manual/[별지 2] 개인정보의 수집&이용 및 제공에 관한 동의서_20231228수정.pdf"
page_contents, text1 = extract_text_and_tables_to_dataframe(path)

# 추출된 페이지 내용 출력
for content in page_contents:
    page_num = content['Page']
    text_content = content['Text']
    tables = content['Tables']
    
    if text_content:
        print(f"Page {page_num} Text:")
        print(text_content)
    
    if tables:
        for table_num, df in enumerate(tables):
            print(f"Page {page_num} Table {table_num + 1} (Markdown format):")
            markdown_table = dataframe_to_markdown(df)
            print("```\n" + markdown_table + "\n```")
    
    print()

# "■|□|▶|※" 패턴을 기준으로 텍스트를 나누기
pattern = r"(▶|※)\s*(.*)"
split_text = split_text_by_pattern(text1, pattern)

# 각 섹션을 JSON 형식으로 변환하여 저장
documents = [{"section": title.strip(), "content": f"{title.strip()} {content.strip()}"} for title, content in split_text]

# JSON 파일로 저장
save_to_json(documents, '오오오.json')

# 결과 출력 (선택 사항)
for doc in documents:
    print(f"제목: {doc['section']}")
    print(f"내용: {doc['content']}")
    print("===================================")

print("JSON 파일로 저장되었습니다.")


Page 1 Text:
■ 국민내일배움카드 운영규정 [별지 제2호서식]
개인정보의 수집ㆍ이용 및 제공에 관한 동의서
▶ 개인정보 수집ㆍ이용에 관한 안내
ㅇ 고용노동부는 직업능력개발계좌 발급 및 지원 등에 관한 사무를 수행
하기 위하여 「국민 평생 직업능력 개발법 시행령」 제52조의2제1항에
따라 주민등록번호 또는 외국인등록번호가 포함된 자료 및 기타 민감정
보를 다음과 같이 직업능력개발정보망(HRD-Net)에 수집ㆍ관리하고 있으며,
「개인정보 보호법」에서 규정하고 있는 책임과 의무를 준수하고 실천합니
다.
수집목적 수집항목 보유기간
1) 직업능력개발계좌 발급 및 훈련 진단‧상
담
2) 비용지원 및 개인별 훈련이력 관리 성명, 주민등록번호, 주소, 이메일주소
3) 일자리정보, 취업지원사업 등 관련정보 (선택), 전화번호 또는 휴대전화번호, 수집일부터
및 서비스의 제공 그 밖에 학력･자격･경력 등 상담 및 계속
4) 정부의 직업능력개발훈련 성과관리 계좌발급․지원에 필요한 민감정보
5) 훈련 만족도 조사 및 제도개선에 필요한
의견조사 실시 등
ㅇ 고용노동부는 개인정보를 수집목적에 필요한 범위에서 적합하게 처리
하며, 정보주체의 사전 동의 없이 개인정보를 제3자에게 제공하지 않
습니다.
ㅇ 직업능력개발계좌를 발급받은 사람(정보주체)은 자신에 관한 개인정보
의 열람 ․ 수정을 요청할 수 있습니다. 다만, 훈련실시 및 비용지원 등
에 관한 정보는 오류가 있는 경우를 제외하고는 수정․삭제를 요청할 수
없습니다.
▶ 개인정보의 제공에 관한 동의
ㅇ 고용노동부는 취업지원 관련 정보 및 서비스 제공 등을 위하여 직업능력개발계
좌를 발급받은 사람(정보주체)으로부터 제공받은 개인정보를 다음과 같이
제공(보유기관으로부터 수집ㆍ제공 포함)합니다.
Page 1 Table 1 (Markdown format):
```
| 수집목적                                 | 수집항목                             | 보유기간   |
|:----------